In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datasets import load_dataset
import plotly.express as px

In [ ]:

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("DDSC/nordic-embedding-training-data")

In [ ]:
type(ds)

In [ ]:
df = ds['train'].to_pandas()

In [ ]:
df.columns

In [ ]:
df.columns

In [ ]:
for c in df.columns:
    print(c)
    df[f'{c}_len'] = df[c].str.len()


In [ ]:
df.describe()

Query is shortest. 

In [ ]:
px.ecdf(df, x = ["query_len", "posive_len", "negative_len"])